In [ ]:
pip install dalib

In [ ]:
import random
import time
import warnings
import sys
import argparse
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as col
from sklearn.manifold import TSNE

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from torch.optim import SGD
import torch.utils.data
from torch.utils.data import DataLoader
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torch.nn.functional as F

sys.path.append('.')
from dalib.modules.domain_discriminator import DomainDiscriminator
from dalib.adaptation.dann import DomainAdversarialLoss, ImageClassifier
import dalib.vision.datasets as datasets
import dalib.vision.models as models
#from tools.utils import AverageMeter, ProgressMeter, accuracy, ForeverDataIterator
#from tools.transforms import ResizeImage
#from tools.lr_scheduler import StepwiseLR

# add library
from torchvision.models import alexnet
import torch.nn as nn
from torch.autograd import Variable

# GPUが使用できるかを確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# RNG
seed = 0
random.seed(seed)
torch.manual_seed(seed)

# Cudnn setting
cudnn.deterministic = False
cudnn.benchmark = True

def Office31_loader(task, batch_size,train:bool,data_dir = "data/office31"):

    # Computed with compute_mean_std.py
    mean_std = {
        'A': {
            'mean': [0.79235494, 0.7862071 , 0.78418255],
            'std':  [0.31496558, 0.3174693 , 0.3193569 ]
        },
        'D': {
            'mean': [0.47086468, 0.44865608, 0.40637794],
            'std':  [0.20395322, 0.19204104, 0.1996422 ]
        },
        'W': {
            'mean': [0.6119875 , 0.6187739 , 0.61730677],
            'std':  [0.25063968, 0.25554898, 0.25773206]
        }
    }

    if train == True:
      data_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_std[task]['mean'],
                                 std=mean_std[task]['std'])
        ])
    else:
      data_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean_std[task]['mean'],
                                 std=mean_std[task]['std'])                                                        
        ])


    dataset = datasets.office31.Office31(root=data_dir, task=task, download=True, transform=data_transform)
    dataset_loader = DataLoader(dataset, batch_size=batch_size, shuffle=train, drop_last=train)
    return dataset,dataset_loader

def coral(source, target):

    d = source.size(1)  # dim vector (number of class)

    source_c = compute_covariance(source)
    target_c = compute_covariance(target)

    loss = torch.sum(torch.mul((source_c - target_c), (source_c - target_c)))

    loss = loss / (4 * d * d)
    return loss


def compute_covariance(input_data):
    """
    Compute Covariance matrix of the input data
    """
    n = input_data.size(0)  # batch_size

    # Check if using gpu or cpu
    if input_data.is_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    id_row = torch.ones(n).resize(1, n).to(device=device)
    sum_column = torch.mm(id_row, input_data)
    mean_column = torch.div(sum_column, n)
    term_mul_2 = torch.mm(mean_column.t(), mean_column)
    d_t_d = torch.mm(input_data.t(), input_data)
    c = torch.add(d_t_d, (-1 * term_mul_2)) * 1 / (n - 1)

    return c

def train(model, train_loader_dict, optimizer, lambda_coral):
  model.train()
  sum_cl = 0.0
  sum_co = 0.0
  sum_to = 0.0

  train_steps = min(len(train_loader_dict['Source']), len(train_loader_dict['Target']))

  for iteration in range(train_steps):
    source_img, source_label = next(iter(train_loader_dict['Source']))
    target_img, _ = next(iter(train_loader_dict['Target']))

    source_img, source_label = source_img.to(device), source_label.to(device)
    target_img = target_img.to(device)

    source_img, source_label = Variable(source_img), Variable(source_label)
    target_img = Variable(target_img)

    optimizer.zero_grad()

    source_out = model(source_img)
    target_out = model(target_img)

    Class_loss = F.cross_entropy(source_out, source_label)
    Coral_loss = coral(source_out, target_out)
    Total_loss = Class_loss + Coral_loss * lambda_coral

    Total_loss.backward()
    optimizer.step()

    sum_cl += Class_loss
    sum_co += Coral_loss
    sum_to += Total_loss
  
  epoch_cl, epoch_co, epoch_to = sum_cl / train_steps, sum_co / train_steps, sum_to / train_steps
  print('class : {:.4f}, coral : {:.4f}, total : {:.4f}'.format(epoch_cl, epoch_co, epoch_to))

  return epoch_cl, epoch_co


def test(model, val_loader_dict,phase):
  model.eval()
  test_loss = 0.0
  correct = 0
  with torch.no_grad():
    for img, label in val_loader_dict[phase]:
      img, label = img.to(device), label.to(device)
      out = model(img)

      # sum up batch loss
      test_loss += F.cross_entropy(out, label, size_average=False).item()
      # get the index of the max log-probability
      _,preds = torch.max(out, 1)
      correct += torch.sum(preds == label.data)
      
    test_loss /= len(val_loader_dict[phase].dataset)
    test_acc = correct / len(val_loader_dict[phase].dataset)

    print('{} loss : {:.4f} acc : {:.4f}'.format(phase, test_loss, test_acc))
  return test_loss, test_acc


def main(num_epochs):
  
  source = 'A'
  target = 'W'

  lr = 1e-3
  decay = 5e-4
  momentum = 0.9

  lambda_coral = 0.75

  batch_size = 128

  # dataset & dataloader
  source_train_dataset, source_train_loader = Office31_loader(task = source, batch_size = batch_size, train = True)
  target_train_dataset, target_train_loader = Office31_loader(task = target, batch_size = batch_size, train = True)

  source_val_dataset, source_val_loader = Office31_loader(task = source, batch_size = batch_size, train = False)
  target_val_dataset, target_val_loader = Office31_loader(task = target, batch_size = batch_size, train = False)

  train_loader_dict ={'Source':source_train_loader, 'Target':target_train_loader}
  val_loader_dict = {'Source':source_val_loader, 'Target':target_val_loader}

  n_classes = len(source_train_loader.dataset.classes)

  # AlexNet
  model = alexnet(pretrained=True)
  model.classifier[6] = nn.Linear(4096, n_classes)
  torch.nn.init.normal_(model.classifier[6].weight, mean=0, std=5e-3)
  model.classifier[6].bias.data.fill_(0.01)

  model = model.to(device)

  # optimizer 
  optimizer = torch.optim.SGD([
        {'params':  model.features.parameters()},
        {'params': model.classifier[:6].parameters()},
        # fc8 -> 7th element (index 6) in the Sequential block
        {'params': model.classifier[6].parameters(), 'lr': 10 * lr}
    ], lr=lr, momentum=momentum)
  
  # log dict
  history = {}
  history['train_class_loss'] = []
  history['train_coral_loss'] = []
  history['test_source_loss'] = []
  history['test_source_acc'] = []
  history['test_target_loss'] = []
  history['test_target_acc'] = []


  for epoch in range(1, num_epochs+1):
    print('-----------------------')
    print('Epoch {}/{}'.format(epoch,num_epochs))

    epoch_cl, epoch_co = train(model, train_loader_dict, optimizer, lambda_coral)
    s_test_loss, s_test_acc = test(model, val_loader_dict,'Source')
    t_test_loss, t_test_acc = test(model, val_loader_dict,'Target')

    history['train_class_loss'].append(epoch_cl)
    history['train_coral_loss'].append(epoch_co)
    history['test_source_loss'].append(s_test_loss)
    history['test_source_acc'].append(s_test_acc)
    history['test_target_loss'].append(t_test_loss)
    history['test_target_acc'].append(t_test_acc)

  return history

def plot_graph(values1, values2, rng, label1, label2):
    plt.plot(range(rng), values1, label=label1)
    plt.plot(range(rng), values2, label=label2)
    plt.legend()
    plt.grid()
    plt.show()  

In [ ]:
num_epochs = 30
history = main(num_epochs)
plot_graph(history['test_source_acc'], history['test_target_acc'],num_epochs,'source_acc','target_acc')